The `selenium.webdriver` module provdes all the WebDriver implementations, I am using the Chrome implementation.
The `Keys` class provides all the keys in the keyboard.

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

Start Chromedriver

In [ ]:
#Create an instance of the Chrome WebDriver
driver = webdriver.Chrome('C:\\Users\\18053\\Desktop\\chromedriver')

#Driver.get to navigate to a given page
driver.get("http://www.caiso.com/TodaysOutlook/Pages/default.aspx")

In [ ]:
import pandas as pd
import time
import datetime
import pprint
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

Set up date range to scrape from- earliest data on website is April 10 2018

In [3]:
start_dt = datetime.datetime(2018,4,10)

end_dt = datetime.datetime(2022,5,5)
#technically today's data is incomplete but since I will be using the range function which is exclusive for the end value, ending today is fine


Create a list of the dates in the format of MM/DD/YYYY

In [4]:
dates = []
for dt in pd.date_range(start_dt, end_dt):
    dates.append(dt.strftime("%m/%d/%Y"))

Print the dates to make sure the for loop worked correctly. Printing them using `PrettyPrinter`

In [6]:
#pp = pprint.PrettyPrinter(indent=4,compact=True)
#pp.pprint(dates)

Define a function to select a given date on the website

In [7]:
def select_date(date,calendar_class):
    """
    input date in 'MM/DD/YYYY' form, and calendar_class as 'demand-date' or 'net-demand-date'
    
    """
    
    inputElement = driver.find_element(by=By.CLASS_NAME, value=calendar_class) 
    inputElement.clear()
    inputElement.send_keys(date)
    inputElement.send_keys(Keys.ENTER)

In [8]:
def download_file(button_id, hidden_button_id = ('','')):
    """Expecting the following:
    button_id: downloadDemandCSV, downloadNetDemandCSV
    Hidden_button_id: (dropdownMenu1,0), (dropdownMenu1,2)
    button_id: actual download button, hidden button is the one hiding it.  Hidden index is optional index for hiding button."""
    
    if hidden_button_id[0] !='':
        if hidden_button_id[1]!='':
            inputElement = driver.find_elements(by=By.ID, value=hidden_button_id[0])[hidden_button_id[1]]
            inputElement.send_keys(Keys.ENTER)
        else:
            inputElement = driver.find_elements(by=By.ID, value=hidden_button_id[0])
            inputElement.send_keys(Keys.ENTER)
    wait = WebDriverWait(driver, 20)
    downloadButton= wait.until(EC.element_to_be_clickable((By.ID,button_id)))
    downloadButton.click()

In [9]:
#hidden_button_id = ('dropdownMenu1',0)
#inputElement = driver.find_elements(by=By.ID, value=hidden_button_id[0])
#inputElement.send_keys(Keys.ENTER)
#print(len(inputElement))

In [10]:
def daterange(date1, date2):
    for n in range(int ((date2 - date1).days)+1):
        yield date1 + datetime.timedelta(n)

In [ ]:
for dt in dates:
    # select_date(dt,'form-control form-control-sm date demand-date')
    select_date(dt,'demand-date')
    print(dt)
    download_file('downloadDemandCSV',('dropdownMenu1',0))
    #select_date(dt,'net-demand-date')
    #download_file('downloadNetDemandCSV',('dropdownMenu1',2))

In [12]:
driver.close()